In [648]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Model
from keras.layers import Activation, Dense,Conv2D,Dropout,Activation,Flatten,MaxPooling2D
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense,
#from tensorflow.keras.utils import to_categorical
import imblearn
from imblearn.over_sampling import RandomOverSampler,SMOTE
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import librosa
from sklearn.model_selection import train_test_split



In [649]:
train_path='D:\\upwork\\New folder (2)\\training'
test_path='D:\\upwork\\New folder (2)\\testing'
model_path='D:\\upwork\\New folder (2)\\audio_classification'


In [650]:
def features_extractorcnn(file_name):
    audio, sample_rate = librosa.load(file_name) 
    mfccs_features =librosa.feature.melspectrogram(y=audio,sr=sample_rate)
    #mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_features.reshape(128,44,1)

In [651]:
features_extractorcnn('D:\\upwork\\New folder (2)\\training\\blue.unknown.2uelmobe.ingestion-5b9bcd5b54-nclkz.wav').shape

(128, 44, 1)

In [653]:
x=np.array([features_extractorcnn(train_path+'\\'+f) for f in listdir(train_path) if isfile(join(train_path, f))])
x_test=np.array([features_extractorcnn(test_path+'\\'+f) for f in listdir(test_path) if isfile(join(test_path, f))])

In [655]:
y=np.array([f.split('.')[0] for f in listdir(train_path) if isfile(join(train_path, f))])
y=np.array(pd.get_dummies(y))

#y_test=np.array([f.split('.')[0] for f in listdir(test_path) if isfile(join(test_path, f))])
#y_test=np.array(pd.get_dummies(y_test))

In [656]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=0)

In [657]:
oversample = RandomOverSampler(sampling_strategy='minority')
#x_over, y_over = oversample.fit_resample(x_train, y_train)
x_over, y_over = oversample.fit_resample(x_train.reshape(len(x_train),-1), y_train)
x_over=x_over.reshape(len(x_over),128,44,1)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [658]:
x_over.shape

(462, 128, 44, 1)

In [673]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model=keras.Sequential([
    #cnn
    layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(128,44,1)),
    keras.layers.Dropout(0.4),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    keras.layers.Dropout(0.4),
    layers.MaxPooling2D((2,2)),
    
    #dense
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    keras.layers.Dropout(0.4),
    layers.Dense(16,activation='softmax')
    
])
model.compile(optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy'])
model.fit(x_over,y_over, epochs=10, validation_data=(x_val, y_val),callbacks=[callback])


Epoch 1/10
15/15 [==============================] - 3s 169ms/step - loss: 14.2628 - accuracy: 0.6082 - val_loss: 5.3128 - val_accuracy: 0.6220
Epoch 2/10
15/15 [==============================] - 2s 165ms/step - loss: 3.9804 - accuracy: 0.6991 - val_loss: 1.9418 - val_accuracy: 0.6463
Epoch 3/10
15/15 [==============================] - 3s 215ms/step - loss: 1.3360 - accuracy: 0.7035 - val_loss: 1.5051 - val_accuracy: 0.7195
Epoch 4/10
15/15 [==============================] - 3s 205ms/step - loss: 0.9424 - accuracy: 0.7208 - val_loss: 1.3622 - val_accuracy: 0.6707
Epoch 5/10
15/15 [==============================] - 3s 209ms/step - loss: 0.7366 - accuracy: 0.7489 - val_loss: 1.3144 - val_accuracy: 0.7439
Epoch 6/10
15/15 [==============================] - 3s 211ms/step - loss: 0.6687 - accuracy: 0.7576 - val_loss: 1.3041 - val_accuracy: 0.7073
Epoch 7/10
15/15 [==============================] - 3s 216ms/step - loss: 0.7350 - accuracy: 0.7727 - val_loss: 1.2604 - val_accuracy: 0.7561
Epoch

In [675]:
model.summary()

Model: "sequential_80"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 126, 42, 32)       320       
_________________________________________________________________
dropout_117 (Dropout)        (None, 126, 42, 32)       0         
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 63, 21, 32)        0         
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 61, 19, 64)        18496     
_________________________________________________________________
dropout_118 (Dropout)        (None, 61, 19, 64)        0         
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 30, 9, 64)         0         
_________________________________________________________________
flatten_46 (Flatten)         (None, 17280)           

In [694]:
model.predict(x_test)

array([[1.57463455e-05, 1.28463525e-02, 2.73213232e-06, ...,
        1.57287926e-04, 4.46613376e-06, 5.51449833e-04],
       [4.45998413e-03, 4.10872288e-02, 5.41455718e-03, ...,
        2.44749989e-03, 3.92155908e-03, 7.30364874e-04],
       [9.82374847e-02, 8.81673861e-03, 1.50404330e-02, ...,
        2.16458738e-03, 2.80724820e-02, 2.71524414e-02],
       ...,
       [5.17129783e-10, 3.96168565e-09, 6.12924295e-12, ...,
        4.44893731e-12, 2.51287324e-10, 9.41983817e-12],
       [4.56418209e-02, 8.75389576e-02, 1.79952048e-02, ...,
        1.90518852e-02, 4.29042280e-02, 2.44367607e-02],
       [2.03563273e-02, 6.16203696e-02, 1.07393675e-02, ...,
        8.94686114e-03, 5.22967381e-03, 6.02099672e-03]], dtype=float32)

In [695]:
predictions=[classes[i] for i in np.argmax(model.predict(x_test.reshape(len(x_test-1),128,44,1)),axis=1)]

In [696]:
test_classes=[f.split('.')[0] for f in listdir(test_path) if isfile(join(test_path, f))]

In [697]:
mydf=pd.DataFrame(data={'pred':predictions,'real':test_classes})

In [698]:
mydf

,pred,real
0,unknown,change_color
1,unknown,change_color
2,unknown,change_color
3,noise,noise
4,noise,noise
...,...,...
82,unknown,unknown
83,unknown,unknown
84,unknown,unknown
85,unknown,unknown
